In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
%matplotlib inline

In [2]:
ej_comments_all = pd.read_json('/tmp/airflow/comments.json')

In [3]:
ej_comments = pd.DataFrame(data=ej_comments_all, columns=['comentário_id', 'comentário', 'concorda', 'discorda', 'pulados'])
ej_comments['concorda'] = ej_comments['concorda'].map(lambda x: x * 100)
ej_comments['discorda'] = ej_comments['discorda'].map(lambda x: x * 100)
ej_comments['pulados'] = ej_comments['pulados'].map(lambda x: x * 100)
ej_comments['compilado'] = ''
for index,value in enumerate(ej_comments['concorda']):
    ej_comments['compilado'][index] = [ej_comments['concorda'][index], ej_comments['discorda'][index], ej_comments['pulados'][index]]
a = ej_comments_all.sort_values(by='discorda', ascending=False)
ej_comments_all[ej_comments_all['participação'] > 0.5]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,comentário,comentário_id,autor,autor_id,concorda,discorda,pulados,convergência,participação,criado
0,Caixa 2 tem que ser crime sim! Se não punir os...,666,ucc@br.transparency.org,1051,0.903226,0.072581,0.024194,0.851240,0.746988,2020-04-06T15:31:42.617Z
1,Tornar o Caixa 2 crime ajudará a evitar essa p...,667,ucc@br.transparency.org,1051,0.908333,0.058333,0.033333,0.879310,0.722892,2020-04-06T15:31:42.630Z
2,O Caixa 2 é crime e muda o resultado das eleiç...,668,ucc@br.transparency.org,1051,0.924370,0.058824,0.016807,0.880342,0.716867,2020-04-06T15:31:42.632Z
3,Acho que a Lei de Abuso de Autoridade é uma mo...,669,ucc@br.transparency.org,1051,0.754098,0.213115,0.032787,0.559322,0.734940,2020-04-06T15:31:42.635Z
4,É preciso muito cuidado com a Lei de Abuso de ...,670,ucc@br.transparency.org,1051,0.888000,0.080000,0.032000,0.834711,0.753012,2020-04-06T15:31:42.637Z
5,"Ninguém pode ficar acima da Lei! Por isso, é p...",671,ucc@br.transparency.org,1051,0.822581,0.129032,0.048387,0.728814,0.746988,2020-04-06T15:31:42.640Z
6,Cadeia por muito tempo e obrigação de devolver...,672,ucc@br.transparency.org,1051,0.854962,0.068702,0.076336,0.851240,0.789157,2020-04-06T15:31:42.644Z
7,Aumentar a pena para casos de corrupção e exig...,673,ucc@br.transparency.org,1051,0.829457,0.147287,0.023256,0.698413,0.777108,2020-04-06T15:31:42.647Z
8,Só poderemos endurecer as penas em casos de co...,674,ucc@br.transparency.org,1051,0.630252,0.336134,0.033613,0.304348,0.716867,2020-04-06T15:31:42.649Z
9,É fundamental que se crie dispositivos para au...,727,1002282-mautic@mail.com,1117,0.935185,0.037037,0.027778,0.923810,0.650602,2020-04-17T14:56:22.670Z


In [10]:
from mautic_sdk import MauticSdk 

def fill_df_with_mautic_email(df):
    df.rename(columns={'criado': 'votos'}, inplace=True)
    mautic_sdk = MauticSdk()
    for ej_email in df['email']:
        mautic_email = mautic_sdk.get_contact_email(ej_email)
        df.loc[df['email'] == ej_email, 'mautic_email'] = mautic_email
    df.fillna("empty", inplace=True)
    return df

votes_df = pd.read_json('/tmp/airflow/ej_only.json')
tmp = votes_df.groupby(['email']).count().reset_index(level=0).reset_index(level=0)
top_50_voters_df = pd.DataFrame(tmp, columns=['criado', 'email']) 
top_50_voters_df.rename(columns={'criado': 'votos'}, inplace=True)
top_50_voters_df.sort_values(by='votos', inplace=True, ascending=False)

top_50_with_mautic_mail = pd.DataFrame(top_50_voters_df)
top_50_with_mautic_mail = fill_df_with_mautic_email(top_50_with_mautic_mail)
top_50_with_mautic_mail

,votos,email,mautic_email
99,32,1020159-mautic@mail.com,simonevilasboas@hotmail.com
98,32,1018116-mautic@mail.com,empty
97,32,1013594-mautic@mail.com,empty
96,32,1013378-mautic@mail.com,empty
88,32,1012568-mautic@mail.com,laeliopereira@hotmail.com
...,...,...,...
95,1,1013290-mautic@mail.com,empty
62,1,1011065-mautic@mail.com,empty
100,1,137549-mautic@mail.com,ricabras@gmail.com
61,1,1011051-mautic@mail.com,empty


34